# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Признаки**

    DateCrawled — дата скачивания анкеты из базы
    VehicleType — тип автомобильного кузова
    RegistrationYear — год регистрации автомобиля
    Gearbox — тип коробки передач
    Power — мощность (л. с.)
    Model — модель автомобиля
    Kilometer — пробег (км)
    RegistrationMonth — месяц регистрации автомобиля
    FuelType — тип топлива
    Brand — марка автомобиля
    Repaired — была машина в ремонте или нет
    DateCreated — дата создания анкеты
    NumberOfPictures — количество фотографий автомобиля
    PostalCode — почтовый индекс владельца анкеты (пользователя)
    LastSeen — дата последней активности пользователя

**Целевой признак**

    Price — цена (евро)

## Подготовка данных

In [1]:
try:
    import pandas_profiling
except:
    !pip install pandas-profiling
    import pandas_profiling

     |████████████████████████████████| 325 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 679 kB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 53.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 65.2 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 82.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 68.5 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27085 sha256=43b4a63f25822cbd8a6385227cb411e8120dd72efab0fada658e0a9ec145815e
  Stored in directory: /home/jovyan/.cache/pip/wheels/1d/05/04/c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
Successfully built htmlmin


In [2]:
from collections import namedtuple

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from pandas.api.types import is_numeric_dtype

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

__________________________

In [3]:
RAND = sum(ord(x) for x in 'NEVER SURRENDER')

____________________

### Загрузка и оценка данных

In [4]:
df_auto = pd.read_csv('/datasets/autos.csv')
df_auto.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


In [5]:
#columns snake_case
df_auto.columns = df_auto.columns.str.replace(r'(.)([A-Z])', r'\1_\2', regex=True).str.lower()

In [6]:
#df_auto.profile_report()

Основные проблемы:
- `number_of_pictures` содержит только одно значение, 0. Признак неинформативный
- есть дубликаты (4 строки)
- пропущенные значения в `vehicle_type`, `gearbox`, `model`, `registration_month`, `fuel_type`, `repaired`
- в `price` есть нулевые значения 
- `registration_year` - аномальные значения 
- `power` - нулевые значения, аномальные значения
- `registration_month` - аномальные значения (13 месяцев), может быть связано с тем, что данные были скачаны из разных баз, в одной из которых месяца обозначаются 0-11, в другой 1-12.

Не все признаки влияют на цену. Перед обработкой данных выделим те, которые могут влиять на цену авто.

**Значимые:**
- vehicle_type   
- registration_year 
- gearbox 
- power 
- model 
- kilometer 
- fuel_type 
- brand 
- repaired 

**Остальные:**
- date_crawled  
- last_seen 
- date_created   
Эти три признака больше для предобработки данных. Т.к. цены на авто меняются со временем, дата скачивания анкеты из базы дает представление, для какого периода данные актуальны (2016 год для наших данных).
Теоретически, может быть полезной информация, сколько времени активно объявление (если машину не покупали очень долго, то цена может быть неадекватна для авто с такими параметрами). Но в данном случае взят небольшой период времени и нет информации о том, какое среднее время продажи для авто должно быть.
- registration_month - избыточен, достаточно года регистрации
- number_of_pictures 
- postal_code - теоретически местонахождение машины так же может влиять на стоимость, но в данном случае будем учитывать только параметры самого авто
   

_____________________

### Предобработка данных

In [7]:
size_all_data = df_auto.shape[0]

Удалим дубликаты

In [8]:
df_auto = df_auto.drop_duplicates().reset_index(drop=True)

Удалим неинформативные признаки

In [9]:
uninformative_col = ['date_crawled',
                     'last_seen',
                     'date_created',
                     'registration_month',
                     'number_of_pictures',
                     'postal_code']

In [10]:
df_auto = df_auto.drop(uninformative_col, axis=1)

________________________
`price` - целевой признак. Нулевую стоимость в данном случае можно считать ошибкой в данных и удалить все записи с нулевой ценой(сюда же включим авто с ценой менее 100). Все остальные записи будем считать верными, сильно выделяющихся значений нет.

In [11]:
df_auto = df_auto[df_auto['price'] > 100]

__________________________________
`registration_year` - установим допустимые рамки для рассматриваемых авто. Пусть это будут авто с годом регистрации позже 1966(не будем рассматривать "антиквариат" старше 50 лет на момент 2016) и не позже 2016 (т.к. объявления были скачаны из базы в 2016 году, год регистрации авто не должен быть больше)

Проверим, сколько записей выходят за рамки

In [12]:
print(f"All: {df_auto.query('registration_year < 1966 or registration_year > 2016')['price'].count()}")
print(f"Over 2016: {df_auto.query('registration_year > 2016')['price'].count()}")
print(f"Less 1966: {df_auto.query('registration_year < 1966')['price'].count()}")

All: 14358
Over 2016: 13673
Less 1966: 685


Найдено 14358 записей, это около 4% от всех данных. 

Записей с годом регистрации менее 1966 года (685 штук) не так много, их можно удалить. 

Авто с регистрацией после 2016 года достаточно много. Нет возможности уточнить у источника данных, с чем это может быть связано. Можно либо принудительно поставить им 2016 год (что может быть привести к ошибкам), либо удалить.

В данном случае удалим все аномальные значения.

In [13]:
df_auto = df_auto.query('registration_year > 1965 and registration_year < 2017')

_______________
`power` - мощность авто. 

Существуют совсем маломощные авто, например "Une Voiture Sans Permis" - авто "без прав", имющее мощность 5 л.с. Для наших данных возьмем именно эти данные за нижнюю границу.

Машины с мощностью более 1000 л.с. хоть и существуют, но выпускаются ограниченными партиями, в данном случае будем считать значения аномальными/ошибочными.

Авто с мощностью более 300 л.с. встречаются не так часто, но все же есть на дорогах, оставим их в данных и поставим "верхнюю границу" мощности в 1000 л.с.

In [14]:
df_auto = df_auto.query('power < 1001 and power > 4')

Обработаны аномальные значения, проверим, какой процент данных остался от изначальных:

In [15]:
df_auto.shape[0] / size_all_data

0.8307244708199645

17% данных удалено, это достаточно много. 

In [16]:
df_auto = df_auto.reset_index(drop=True)

________________________________
Теперь попробуем заполнить пропуски

`model` - по имеющимся данным достовено восстановить модель авто сложно, заменим пропуски на 'other'

In [17]:
df_auto['model'] = df_auto['model'].fillna('other')

`gearbox` - так же нет возможности восстановить по имеющимся данным (даже одинаковая модель авто может иметь разный тип коробки передач)

`repaired` - нет возможности восстановить

Пропущенные значения заменим на 'unknown'

In [18]:
df_auto['gearbox'] = df_auto['gearbox'].fillna('unknown')

In [19]:
df_auto['repaired'] = df_auto['repaired'].fillna('unknown')

`vehicle_type` -  по идее, `model` должно однозначно определять тип кузова авто. Но т.к. у нас есть модели `other` и есть вероятность, что одна модель "принадлежит" двум маркам авто - будем определять тип автомобильного кузова по модели и марке авто

`fuel_type` - аналогично `vehicle_type`

In [20]:
def fill_date(data: pd.DataFrame, 
              target: str, 
              groupby_: list='') -> pd.DataFrame:
    return data.groupby(groupby_)[target].transform(lambda x: x.fillna(x.mode()[0]))
    

In [21]:
df_auto['vehicle_type'] = fill_date(df_auto, 'vehicle_type',['model','brand'])

In [22]:
df_auto['fuel_type'] = fill_date(df_auto, 'fuel_type',['model','brand'])

Все обозначенные проблемы обработаны, посмотрим сокращенный отчет по получившимся данным:

In [23]:
#df_auto.profile_report(minimal=True)

Все пропущенные значения заменены.

Проверим данные еще раз на дубликаты

In [24]:
df_auto.duplicated().sum()

40731

Появились дубликаты, но т.к. в данных больше нет сведений, которые говорят про уникальность записи (например, время размещения объявления) - будем считать эти дубликаты разными машинами и оставим их.

### Вывод


- данные были загружены и изучены
- выбраны необходимые для исследования параметры
- удалены аномальные значения
- в результате убрано 17% "поврежденных" данных
- обработаны пропущенные значения

## Обучение моделей

**Используемые функции**

In [25]:
def data_split(data: pd.DataFrame,
               target_: str = 'price',
               test_size_: float = 0.25) -> [namedtuple, namedtuple]:
    #create namedtuples for sampling
    train = namedtuple('features','target')
    test = namedtuple('features','target')
    
    train.features, test.features, train.target, test.target = train_test_split(data.drop(target_,axis=1), 
                                                                                df_auto[target_], 
                                                                                test_size=test_size_,   
                                                                                random_state=RAND)
    
    print(f"Size of train {len(train.features)}, percentage of total {len(train.features)/len(df_auto['price']):.0%}")
    print(f"Size of test {len(test.features)}, percentage of total {len(test.features)/len(df_auto['price']):.0%}\n")
    return train, test
    

In [26]:
data_process = {}
data_process_test = {}

In [27]:
def create_model(model_name: str, 
                 data_process_: str,
                 data: namedtuple = None, 
                 parametrs: dict = {},
                 **kwargs): 
    #get data 
    if not data:
        try:
            data = data_process[data_process_]
        except:
            return 'No data'
    
    #create model
    model = eval(model_name)(**kwargs)
    
    #use GridSearchCV
    grid = GridSearchCV(model, parametrs, cv=3, scoring='neg_root_mean_squared_error')
    grid.fit(data.features, data.target)
    
    return [model_name, data_process_, 
            abs(grid.best_score_), 
            grid.best_params_, 
            grid.refit_time_, 
            grid.best_estimator_]

_____________________
Разобьем данные на 2 выборки: тренировночная и тестовая.

У моделей есть разные требования к исходных данным, сделаем несколько наборов данных:
- `raw` без обработки
- `ohe` с OHE кодированием
- `standart` с OHE кодирование и стандартизацией
- `category` с типом "category" для категориальных признаков
- `ordinal` Ordinal кодирование

**Raw**

In [28]:
%%time
#without OHE
train, test = data_split(df_auto)
data_process['raw'] = train
data_process_test['raw'] = test

Size of train 220787, percentage of total 75%
Size of test 73596, percentage of total 25%

CPU times: user 68.2 ms, sys: 3.94 ms, total: 72.1 ms
Wall time: 78.4 ms


In [29]:
# name of categorial and numeric columns
cat_col = [col for col in train.features.columns if not is_numeric_dtype(train.features[col])]
num_col = train.features.columns[~train.features.columns.isin(cat_col)].tolist()

**Category**

In [30]:
%%time
#cateroty dtype
for col in cat_col:
    train.features[col] = pd.Series(train.features[col], dtype="category")
    test.features[col] = pd.Series(test.features[col], dtype="category")
data_process['category'] = train
data_process_test['category'] = test

CPU times: user 137 ms, sys: 0 ns, total: 137 ms
Wall time: 147 ms


**OHE**

In [31]:
%%time
#with OHE
train, test = data_split(pd.get_dummies(df_auto, drop_first=True))
data_process['ohe'] = train
data_process_test['ohe'] = test

Size of train 220787, percentage of total 75%
Size of test 73596, percentage of total 25%

CPU times: user 977 ms, sys: 199 ms, total: 1.18 s
Wall time: 1.19 s


**Standart**

In [32]:
%%time
#OHE and standart
scaler = StandardScaler().fit(train.features[num_col]) 
train.features[num_col] = scaler.transform(train.features[num_col])
test.features[num_col] = scaler.transform(test.features[num_col])
data_process['standart'] = train
data_process_test['standart'] = test

CPU times: user 127 ms, sys: 11.5 ms, total: 138 ms
Wall time: 144 ms


In [33]:
del scaler

**Ordinal**

In [34]:
%%time
#Ordinal
enc = OrdinalEncoder().fit(df_auto[cat_col])
df_auto[cat_col] = enc.transform(df_auto[cat_col])
train, test = data_split(df_auto)
data_process['ordinal'] = train
data_process_test['ordinal'] = test

Size of train 220787, percentage of total 75%
Size of test 73596, percentage of total 25%

CPU times: user 894 ms, sys: 33.4 ms, total: 928 ms
Wall time: 937 ms


In [35]:
#whos

In [36]:
del df_auto
del test
del train
del size_all_data
del uninformative_col

___________________________
Будем использовать:
- DecisionTreeRegressor
- LinearRegression
- RandomForestRegressor
- CatBoostRegressor
- LGBMRegressor

Для подбора гиперпараметров будем использовать GridSearchCV - нет необходимости выделять валидационную выборку. Подбор модели по метрике RMSE.

Для каждой модели будем брать небольшой набор гиперпараметров для экономии времени.

"Простые" модели (Линейная Регрессия, Случайный лес, Дерево) не умеют работать с категориальными переменными без кодирования.

In [37]:
models = []

### LinearRegression

- `ohe` с OHE кодированием
- `standart` с OHE кодирование и стандартизацией

In [38]:
%%time
models.append(create_model('LinearRegression', 'standart'))

CPU times: user 46.1 s, sys: 22.1 s, total: 1min 8s
Wall time: 1min 8s


In [39]:
%%time
models.append(create_model('LinearRegression', 'ohe'))

CPU times: user 45.2 s, sys: 19.4 s, total: 1min 4s
Wall time: 1min 5s


In [40]:
del data_process['standart']

### DecisionTreeRegressor

- `ohe` с OHE кодированием
- `ordinal` Ordinal кодирование

In [41]:
%%time
dtr_params = {'max_depth': [2,7], 
              'min_samples_split': [10, 50],
              'random_state': [RAND]}

models.append(create_model('DecisionTreeRegressor', 
                           'ohe',
                           parametrs=dtr_params))

CPU times: user 19.4 s, sys: 3.52 s, total: 22.9 s
Wall time: 23 s


In [42]:
%%time
models.append(create_model('DecisionTreeRegressor', 
                           'ordinal',
                           parametrs=dtr_params))

CPU times: user 2.1 s, sys: 31.4 ms, total: 2.13 s
Wall time: 2.14 s


### RandomForestRegressor

- `ohe` с OHE кодированием
- `ordinal` Ordinal кодирование

In [43]:
%%time
rfr_params = {'max_depth': [2,7], 
              'min_samples_split': [10, 50],
              'n_estimators': [2, 10],
              'random_state': [RAND]}

models.append(create_model('RandomForestRegressor', 
                           'ohe',       
                           parametrs=rfr_params))

CPU times: user 2min 20s, sys: 7.6 s, total: 2min 28s
Wall time: 2min 28s


In [44]:
%%time
models.append(create_model('RandomForestRegressor', 
                           'ordinal',       
                           parametrs=rfr_params))

CPU times: user 18 s, sys: 148 ms, total: 18.2 s
Wall time: 18.2 s


### CatBoostRegressor

- `raw` без обработки
- `ohe` с OHE кодированием
- `ordinal` Ordinal кодирование

In [45]:
%%time
cbr_params = {'learning_rate': [0.1,0.4], 
              'iterations': [50, 100],
              'random_state': [RAND],
              'verbose' : [100],
             }

models.append(create_model('CatBoostRegressor', 
                           'raw', 
                           parametrs=cbr_params,   
                           cat_features=cat_col))

0:	learn: 4308.0780145	total: 122ms	remaining: 5.97s
49:	learn: 1863.3445876	total: 3.08s	remaining: 0us
0:	learn: 4311.9552898	total: 65.5ms	remaining: 3.21s
49:	learn: 1857.0182270	total: 2.88s	remaining: 0us
0:	learn: 4308.3989054	total: 78.6ms	remaining: 3.85s
49:	learn: 1855.9094349	total: 3.01s	remaining: 0us
0:	learn: 3489.9978380	total: 66.9ms	remaining: 3.28s
49:	learn: 1696.2592075	total: 2.85s	remaining: 0us
0:	learn: 3491.2456482	total: 65.2ms	remaining: 3.2s
49:	learn: 1683.6844096	total: 2.94s	remaining: 0us
0:	learn: 3515.1982720	total: 77.4ms	remaining: 3.79s
49:	learn: 1696.3539948	total: 2.97s	remaining: 0us
0:	learn: 4308.0780145	total: 64.7ms	remaining: 6.4s
99:	learn: 1758.4298722	total: 5.67s	remaining: 0us
0:	learn: 4311.9552898	total: 72.8ms	remaining: 7.21s
99:	learn: 1750.0932488	total: 5.63s	remaining: 0us
0:	learn: 4308.3989054	total: 72ms	remaining: 7.13s
99:	learn: 1749.8592596	total: 5.75s	remaining: 0us
0:	learn: 3489.9978380	total: 63ms	remaining: 6.23s

In [46]:
%%time
models.append(create_model('CatBoostRegressor', 
                          'ordinal',
                           parametrs=cbr_params))

0:	learn: 4315.2008100	total: 24.3ms	remaining: 1.19s
49:	learn: 1915.9556888	total: 993ms	remaining: 0us
0:	learn: 4317.7511832	total: 19.7ms	remaining: 964ms
49:	learn: 1902.5617588	total: 996ms	remaining: 0us
0:	learn: 4308.7840733	total: 20.6ms	remaining: 1.01s
49:	learn: 1909.9999944	total: 997ms	remaining: 0us
0:	learn: 3519.4915484	total: 20.9ms	remaining: 1.02s
49:	learn: 1705.1516193	total: 1.03s	remaining: 0us
0:	learn: 3515.2162944	total: 27.6ms	remaining: 1.35s
49:	learn: 1700.9244169	total: 1.14s	remaining: 0us
0:	learn: 3516.8129689	total: 22.4ms	remaining: 1.09s
49:	learn: 1703.3338285	total: 1.03s	remaining: 0us
0:	learn: 4315.2008100	total: 20.6ms	remaining: 2.04s
99:	learn: 1788.2848720	total: 2.05s	remaining: 0us
0:	learn: 4317.7511832	total: 32.3ms	remaining: 3.19s
99:	learn: 1775.5119424	total: 2.03s	remaining: 0us
0:	learn: 4308.7840733	total: 21.7ms	remaining: 2.15s
99:	learn: 1778.8245833	total: 2.12s	remaining: 0us
0:	learn: 3519.4915484	total: 22.3ms	remaining

In [47]:
%%time
models.append(create_model('CatBoostRegressor', 
                          'ohe',
                           parametrs=cbr_params))

0:	learn: 4311.0204702	total: 23.4ms	remaining: 1.15s
49:	learn: 1904.7315455	total: 1.04s	remaining: 0us
0:	learn: 4312.1011079	total: 21ms	remaining: 1.03s
49:	learn: 1898.0097927	total: 1.15s	remaining: 0us
0:	learn: 4297.0636343	total: 22.6ms	remaining: 1.11s
49:	learn: 1897.9615181	total: 1.11s	remaining: 0us
0:	learn: 3502.1200223	total: 20.1ms	remaining: 983ms
49:	learn: 1706.6612890	total: 947ms	remaining: 0us
0:	learn: 3491.7810673	total: 20.2ms	remaining: 991ms
49:	learn: 1690.7578574	total: 998ms	remaining: 0us
0:	learn: 3468.1212510	total: 20.2ms	remaining: 990ms
49:	learn: 1702.2135977	total: 968ms	remaining: 0us
0:	learn: 4311.0204702	total: 25.8ms	remaining: 2.55s
99:	learn: 1784.4861186	total: 2.03s	remaining: 0us
0:	learn: 4312.1011079	total: 20.7ms	remaining: 2.05s
99:	learn: 1776.7767528	total: 2.12s	remaining: 0us
0:	learn: 4297.0636343	total: 20.8ms	remaining: 2.05s
99:	learn: 1777.4592591	total: 2.06s	remaining: 0us
0:	learn: 3502.1200223	total: 21.9ms	remaining: 

In [48]:
del data_process['raw']

### LGBMRegressor

- `ordinal` Ordinal кодирование
- `category` с типом "category" для категориальных признаков
- `ohe` с OHE кодированием

In [49]:
%%time
lgbm_params = {'num_leaves':[50,80],               
               'random_state': [RAND]}

   
models.append(create_model('LGBMRegressor', 
                           'category',
                           parametrs=lgbm_params))

CPU times: user 13min 34s, sys: 6.37 s, total: 13min 41s
Wall time: 13min 48s


In [50]:
%%time
models.append(create_model('LGBMRegressor', 
                           'ordinal',
                           parametrs=lgbm_params))

CPU times: user 2min 48s, sys: 959 ms, total: 2min 49s
Wall time: 2min 51s


In [51]:
%%time
models.append(create_model('LGBMRegressor', 
                           'ohe',
                           parametrs=lgbm_params))

CPU times: user 1min 3s, sys: 2.11 s, total: 1min 5s
Wall time: 1min 6s


### Вывод
Был произведен подбор гиперпараметров для нескольких выбранных моделей. Данные по моделям, собраны в табличку и готовы к дальнейшей обработке.

In [52]:
pd.DataFrame(models)

,0,1,2,3,4,5
0,LinearRegression,standart,2607.677942,{},19.583253,LinearRegression()
1,LinearRegression,ohe,2607.677942,{},20.081945,LinearRegression()
2,DecisionTreeRegressor,ohe,2166.680174,"{'max_depth': 7, 'min_samples_split': 50, 'ran...",3.277531,"DecisionTreeRegressor(max_depth=7, min_samples..."
3,DecisionTreeRegressor,ordinal,2170.888065,"{'max_depth': 7, 'min_samples_split': 50, 'ran...",0.326765,"DecisionTreeRegressor(max_depth=7, min_samples..."
4,RandomForestRegressor,ohe,2106.451427,"{'max_depth': 7, 'min_samples_split': 10, 'n_e...",20.974201,"(DecisionTreeRegressor(max_depth=7, max_featur..."
5,RandomForestRegressor,ordinal,2105.994756,"{'max_depth': 7, 'min_samples_split': 10, 'n_e...",2.490304,"(DecisionTreeRegressor(max_depth=7, max_featur..."
6,CatBoostRegressor,raw,1660.381393,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",9.377119,<catboost.core.CatBoostRegressor object at 0x7...
7,CatBoostRegressor,ordinal,1666.745414,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",4.094122,<catboost.core.CatBoostRegressor object at 0x7...
8,CatBoostRegressor,ohe,1654.850813,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",5.040901,<catboost.core.CatBoostRegressor object at 0x7...
9,LGBMRegressor,category,1553.837430,"{'num_leaves': 80, 'random_state': 1114}",59.378307,"LGBMRegressor(num_leaves=80, random_state=1114)"


## Анализ моделей

Заказчику важны:
- качество предсказания
- скорость предсказания
- время обучения

Внесем в сводную таблицу данные по времени предсказания:

In [53]:
%%time
pred = []
for i, modl in enumerate(models):
    ### train time move to create_model()
    #models [-1] - model
    pred.append(modl[-1].predict(data_process_test.get(modl[1]).features))
    pred_time = %timeit -n1 -r1 -o modl[-1].predict(data_process_test.get(modl[1]).features) #-o - output
    models[i][-1] = pred_time.average


197 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
287 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
126 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
7.81 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
166 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
45.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
57.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
13.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
39.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
796 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
CPU times: user 6.78 s, sys: 1.06 s, total: 7.85 s
Wall time: 7.85 s


Оценим полученные значения.

In [54]:
pd.DataFrame(models, columns=['Model', 'Data note', 'RMSE', 'Hyperparams','Train time', 'Predict time'])

,Model,Data note,RMSE,Hyperparams,Train time,Predict time
0,LinearRegression,standart,2607.677942,{},19.583253,0.197400
1,LinearRegression,ohe,2607.677942,{},20.081945,0.286636
2,DecisionTreeRegressor,ohe,2166.680174,"{'max_depth': 7, 'min_samples_split': 50, 'ran...",3.277531,0.125792
3,DecisionTreeRegressor,ordinal,2170.888065,"{'max_depth': 7, 'min_samples_split': 50, 'ran...",0.326765,0.007810
4,RandomForestRegressor,ohe,2106.451427,"{'max_depth': 7, 'min_samples_split': 10, 'n_e...",20.974201,0.166026
5,RandomForestRegressor,ordinal,2105.994756,"{'max_depth': 7, 'min_samples_split': 10, 'n_e...",2.490304,0.045266
6,CatBoostRegressor,raw,1660.381393,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",9.377119,0.057072
7,CatBoostRegressor,ordinal,1666.745414,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",4.094122,0.013795
8,CatBoostRegressor,ohe,1654.850813,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",5.040901,0.039564
9,LGBMRegressor,category,1553.837430,"{'num_leaves': 80, 'random_state': 1114}",59.378307,1.092702


**CatBoostRegressor** и **LGBMRegressor** показали лучшие результаты по метрике RMSE. Лучшие результаты для необработанных данных, но время обучения больше, чем для обработанных.

Для обработанных данных у **LGBMRegressor** все значения RMSE несколько лучше, но время обучения модели почти в 2 раза больше, чем аналогичный результат у **CatBoostRegressor**.

В данном случае все варианты этих моделей хорошие и можно выбирать в зависимости от требований заказчика. Предположим, что нам немного важнее время и как итоговую модель выбираем **CatBoostRegressor** для ordinal-данных.

Проверим выбранную модель на тестовых данных

In [55]:
mean_squared_error(pred[-5], data_process_test['ordinal'].target)**.5

1649.531491742484

**RMSE** модели на тестовых данных 1649.53 - это хороший результат. Значение чуть ниже, чем на тренировочных данных => можно ожидать, что модель не переобучилась.

**Проверка модели на адекватность:**

In [56]:
dummy_mean = DummyRegressor(strategy = 'mean').fit(data_process['ordinal'].features, data_process['ordinal'].target)
dummy_pred = dummy_mean.predict(data_process_test['ordinal'].features)
print(f"Dummy RMSE {mean_squared_error(data_process_test['ordinal'].target, dummy_pred) **.5}")

Dummy RMSE 4601.240832886387


Метрика выбранной модели значительно превышает метрику "наивной модели", значит, модель адекватна.

**Вывод**

- было замерено время, необходимое для работы каждой из моделей
- проведен анализ и выбрана лучшая модель
- оценено качество модели на тестовый данных
- проведена проверка на адекватность

## Вывод

На первом этапе были получены и обработаны данные:
- выделены необходимые для построения модели параметры
- обработаны пропуски и аномалии
____________

Далее было отобрано несколько вариантов для построения моделей
- DecisionTreeRegressor
- LinearRegression
- RandomForestRegressor
- CatBoostRegressor
- LGBMRegressor


По каждой из них произведен поиск по небольшому набору гиперпараметров и отобраны лучшие модели.
_________________

Для выбранных моделей было оценено время, необходимое для обучения модели и для предсказаний.

В результате появилось два лидера:

In [57]:
pd.DataFrame(models, columns=['Model', 'Data note', 'RMSE', 'Hyperparams','Train time', 'Predict time']).iloc[[7,9]]

,Model,Data note,RMSE,Hyperparams,Train time,Predict time
7,CatBoostRegressor,ordinal,1666.745414,"{'iterations': 100, 'learning_rate': 0.4, 'ran...",4.094122,0.013795
9,LGBMRegressor,category,1553.837430,"{'num_leaves': 80, 'random_state': 1114}",59.378307,1.092702


каждого из которых можно рекомендовать, как модель для внедрения.

Было предположено, что время чуть важнее точности и выбрана модель **CatBoostRegressor** с гиперпараметрами {'iterations': 100, 'learning_rate': 0.4}.

Она показала хорошие результаты в том числе и на тестовой выборке (RMSE = 1649.53) и является адекватной.

Таким образом, модель можно рекомендовать для внедрения.